# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("..\\WeatherPy\\output data\\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sorong,-0.8833,131.2500,26.73,83,100,3.15,ID,1742427448
1,1,kruisfontein,-34.0033,24.7314,23.33,38,0,2.33,ZA,1742427449
2,2,ushuaia,-54.8000,-68.3000,7.81,76,75,12.86,AR,1742427373
3,3,mafinga,-7.2500,35.0667,17.97,95,83,0.97,TZ,1742427452
4,4,vilhena,-12.7406,-60.1458,22.12,100,75,4.12,BR,1742427453


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_1= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    hover_cols=["City", "Humidity"],
    color="City"
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_cities_df=city_data_df[
    (city_data_df["Max Temp"]<34) &
    (city_data_df["Max Temp"]>6) &
    (city_data_df["Wind Speed"]<3) &
    (city_data_df["Cloudiness"]<=50)
].dropna()

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
narrow_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,kruisfontein,-34.0033,24.7314,23.33,38,0,2.33,ZA,1742427449
19,19,illizi,26.4833,8.4667,24.65,17,46,2.06,DZ,1742427470
24,24,buka,40.8108,69.1986,14.57,26,0,0.57,UZ,1742427476
28,28,invercargill,-46.4000,168.3500,18.34,68,6,1.79,NZ,1742427481
52,52,papatowai,-46.5619,169.4708,18.29,55,1,2.99,NZ,1742427510
...,...,...,...,...,...,...,...,...,...,...
537,537,plettenberg bay,-34.0527,23.3716,22.32,55,1,1.48,ZA,1742427874
548,548,trang,7.5000,99.5000,24.97,94,20,1.03,TH,1742428083
560,560,cururupu,-1.8283,-44.8683,22.86,99,45,0.56,BR,1742428105
566,566,munnar,10.1000,77.0667,13.81,94,42,2.16,IN,1742428112


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= narrow_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,kruisfontein,ZA,-34.0033,24.7314,38,
19,illizi,DZ,26.4833,8.4667,17,
24,buka,UZ,40.8108,69.1986,26,
28,invercargill,NZ,-46.4000,168.3500,68,
52,papatowai,NZ,-46.5619,169.4708,55,
...,...,...,...,...,...,...
537,plettenberg bay,ZA,-34.0527,23.3716,55,
548,trang,TH,7.5000,99.5000,94,
560,cururupu,BR,-1.8283,-44.8683,99,
566,munnar,IN,10.1000,77.0667,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row["Lat"]
    lng=row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
   

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kruisfontein - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
buka - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
papatowai - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
port blair - nearest hotel: Andaman Ocean Breeze
bredasdorp - nearest hotel: Victoria Hotel
vila velha - nearest hotel: Hotel Prainha
queenstown - nearest hotel: Queens Hotel
ghormach - nearest hotel: No hotel found
jatiroto - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
veinticinco de mayo - nearest hotel: Jorly
tura - nearest hotel: Hotel Polo Orchid
margaret river - nearest hotel: Margaret River Hotel
port elizabeth - nearest hotel: Waterford Hotel
lamarque - nearest hotel: No hotel found
sedhiou - nearest hotel: Hôtel Chez Nous
veigne - nearest hotel: Logis du Lièvre d'OR
udaipur - nearest hotel: Naren Villa
sarrebourg - nearest hotel: Mei
guerrero negro - nearest hotel: Plaza Salparaíso
saint-pier

,City,Country,Lat,Lng,Humidity,Hotel Name
1,kruisfontein,ZA,-34.0033,24.7314,38,No hotel found
19,illizi,DZ,26.4833,8.4667,17,بوناقة
24,buka,UZ,40.8108,69.1986,26,No hotel found
28,invercargill,NZ,-46.4000,168.3500,68,The Grand
52,papatowai,NZ,-46.5619,169.4708,55,No hotel found
...,...,...,...,...,...,...
537,plettenberg bay,ZA,-34.0527,23.3716,55,Milkwood Manor
548,trang,TH,7.5000,99.5000,94,No hotel found
560,cururupu,BR,-1.8283,-44.8683,99,No hotel found
566,munnar,IN,10.1000,77.0667,94,Ammukutty's Cottage


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    hover_cols= ['Hotel Name','country'],
    color="Hotel Name"
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity)